In [ ]:
import inspect
from typing import Any

import keras
import tensorflow as tf
from pydantic import create_model, BaseModel

import stock
import gnn

In [ ]:
m = tf.keras.metrics.CategoricalAccuracy()
m([[0, 1, 0], [0, 1, 0]], [[0, 1, 0], [1, 0, 0]])

In [ ]:
m([[1, 0]], [[1, 1]])

In [ ]:
m([[0, 1]], [[0, 0]])

In [ ]:
json_path = stock.PROJECT_ROOT / "params/sample_image.json"
output_root_dir = stock.PROJECT_ROOT / "results" / json_path.stem
params = gnn.trainer.Trainer.Params.load_json(json_path)
callbacks = gnn.callbacks.CallbackList([
    gnn.callbacks.Checkpoint(output_dir=output_root_dir / "checkpoint"),
    gnn.callbacks.TrainLogger(tensorboard_dir=output_root_dir / "results/tensorboard" / json_path.stem),
])
trainer = gnn.trainer.Trainer(params, callbacks)

In [ ]:
trainer.train()

In [ ]:
params = gnn.trainer.Trainer.Params.get_default_params(
    network_name="resnet50", 
    optimizer_name="adam",
    loss_name="cce",
    dataloader_name="stock_image_dataloader"
)

In [ ]:
print(params.model_dump_json(indent=4))

In [ ]:
y_true = tf.convert_to_tensor([[0], [1], [0], [0]])
y_pred = tf.convert_to_tensor([[0.1], [0.3], [0.4], [0.2]])
bce = keras.losses.BinaryCrossentropy(from_logits=False, reduction="none")
print(bce(y_true, y_pred))

y_true = tf.convert_to_tensor([[0, 1, 0, 0]])
y_pred = tf.convert_to_tensor([[0.1, 0.3, 0.4, 0.2]])
cce = keras.losses.CategoricalCrossentropy(from_logits=False, reduction="none")
print(cce(y_true, y_pred))

In [ ]:
y_true = tf.convert_to_tensor([[0], [1], [0], [0]])
y_pred = tf.convert_to_tensor([[0.1], [0.3], [0.4], [0.2]])
bce = keras.losses.BinaryCrossentropy(from_logits=True, reduction="none")
print(bce(y_true, y_pred))

y_true = tf.convert_to_tensor([[0], [1], [0], [0]])
y_pred = tf.nn.sigmoid(tf.convert_to_tensor([[0.1], [0.3], [0.4], [0.2]]))
bce = keras.losses.BinaryCrossentropy(from_logits=False, reduction="none")
print(bce(y_true, y_pred))

In [ ]:
y_true = tf.convert_to_tensor([[0], [1], [0], [0]])
y_pred = tf.convert_to_tensor([[0.1], [0.3], [0.4], [0.2]])
bce = keras.losses.BinaryCrossentropy(from_logits=False, reduction="sum")
print(bce(y_true, y_pred))

y_true = tf.convert_to_tensor([0, 1, 0, 0])
y_pred = tf.convert_to_tensor([0.1, 0.3, 0.4, 0.2])
bce = keras.losses.BinaryCrossentropy(from_logits=False, reduction="none")
print(bce(y_true, y_pred))

In [ ]:
# y_true = tf.convert_to_tensor([[0], [1], [0], [0]])
# y_pred = tf.convert_to_tensor([[0.1], [0.3], [0.4], [0.2]])
# bce = keras.losses.BinaryCrossentropy(from_logits=False, reduction="none")
# print(bce(y_true, y_pred))

y_true = tf.convert_to_tensor([[0, 1, 0, 0]])
y_pred = tf.convert_to_tensor([[0.1, 0.3, 0.4, 0.2]])
cce = keras.losses.CategoricalCrossentropy(from_logits=True)
print(cce(y_true, y_pred))

y_true = tf.convert_to_tensor([[0, 1, 0, 0]])
y_pred = tf.nn.softmax(tf.convert_to_tensor([[0.1, 0.3, 0.4, 0.2]]))
cce = keras.losses.CategoricalCrossentropy(from_logits=False, reduction="none")
print(cce(y_true, y_pred))

In [ ]:
gnn.trainer.Trainer.Params.get_default_params(
    network_name="resnet50",
    optimizer_name="adam",
    loss_name=""
)

In [ ]:
params = gnn.layers.NetworkParams(
    name="resnet50",
    params={}
)
model = gnn.layers.get_model(params)

In [ ]:
params_class_name = tf.keras.applications.ResNet50.__code__.co_name + "Params"

In [ ]:
def get_network_class(network: tf.keras.Layer):
    param_class_name = network.__code__.co_name + "Params"
    model_param_class = create_model(
        param_class_name,
        **{key: (Any, value.default) if value.default is not inspect.Parameter.empty else (Any, ...) for key, value in inspect.signature(network).parameters.items()}
    )

    class NetworkWrapper(BaseNetwork):

        class Params(model_param_class):
            pass

        def __init__(self, params: Params):
            self.network = network(**params.model_dump())

        def call(self, *args, **kwargs):
            self.network(*args, **kwargs)

    return NetworkWrapper

In [ ]:
model_class = get_network_class(keras.applications.ResNet50)

In [ ]:
model_class()